In [181]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import numpy as np

from selenium.webdriver.common.by import By
from selenium import webdriver

In [182]:
postfix = "with_tags"

#### https://silkwaychina.ru/news/

request only

In [183]:
data = pd.DataFrame()

source = 'silkwaychina'

for year in list(range(2023, 2025)):

    try:
        # Каких-то страниц может не существовать
        for page_num in range(1, 5):
            
            base_url = f'https://silkwaychina.ru/news/?year={year}&PAGEN_1={page_num}'
        
            r = requests.get(base_url)
            
            soup_all = BeautifulSoup(r.text, 'lxml')
    
            items = soup_all.find_all('div', class_=['item col-md-3 col-sm-4 col-xs-6'])
            
            for item in items:

                date = item.find("span", class_="date font_xs").text
                
                title = item.find("div", class_="title font_md").text.strip()
                
                link = item.find("a")["href"]
            
                href = f'https://silkwaychina.ru/news{link}'
            
                r = requests.get(href)
                
                soup = BeautifulSoup(r.text, 'lxml')
                
                content = soup.find('div', class_=['content'])
                
                text = ''.join([i.text.replace('• ', '').replace('\xa0', '').replace('\r\n\t ', '') for i in content.find_all("p")])
                
                text_with_tags = ''.join([str(i) for i in content.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])])
                
                date = soup.find('div', class_=['detail news fixed_wrapper']).find('span', class_='date').text
                
                sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])
            
                data = pd.concat([data, sub_data], axis=0)
                
                time.sleep(1)
                
    except Exception as e:
        print(e)

data.to_excel(f'{source}{postfix}.xlsx')

'NoneType' object has no attribute 'find_all'


In [184]:
print(data.shape) 
display(data)

(80, 6)


,source,title,date,href,text,text_with_tags
0,silkwaychina,Самая ПОЛНАЯ Инструкция по сайту 1688! Товары ...,28 декабря 2023,https://silkwaychina.ru/news/news/aktualnoe/sa...,,
0,silkwaychina,Торговля на OZON и Wildberries: Как начать про...,27 декабря 2023,https://silkwaychina.ru/news/news/statya/torgo...,В данной статье мы рассмотрим ключевые аспект...,<p> В данной статье мы рассмотрим ключевые асп...
0,silkwaychina,Оптимизация процесса доставки товаров с 1688 д...,20 декабря 2023,https://silkwaychina.ru/news/news/statya/optim...,В данной статье рассмотрим практические советы...,<p>В данной статье рассмотрим практические сов...
0,silkwaychina,Китайский Новый год: подготовьтесь к праздничн...,19 декабря 2023,https://silkwaychina.ru/news/news/aktualnoe/ki...,"Скоро начинается Китайский Новый год, а это зн...","<p>Скоро начинается Китайский Новый год, а это..."
0,silkwaychina,Сравнительный анализ: преимущества и недостатк...,15 декабря 2023,https://silkwaychina.ru/news/news/statya/sravn...,В современном мире онлайн-торговли одним из кл...,<p>В современном мире онлайн-торговли одним из...
...,...,...,...,...,...,...
0,silkwaychina,10 трендовых товаров на лето из Китая 2024,28 января 2024,https://silkwaychina.ru/news/news/statya/10-tr...,Узнайте о самых модных и популярных товарах из...,<p>Узнайте о самых модных и популярных товарах...
0,silkwaychina,11 необычных фактов о Китае,21 января 2024,https://silkwaychina.ru/news/news/statya/11-ne...,11 необычных и даже странных фактов о Китае - ...,<p>11 необычных и даже странных фактов о Китае...
0,silkwaychina,Как организовать сопровождение грузоперевозок ...,16 января 2024,https://silkwaychina.ru/news/news/statya/kak-o...,Сегодня многие компании ориентируются на поста...,<p>Сегодня многие компании ориентируются на по...
0,silkwaychina,"Все, что вам нужно знать о доставке из Китая",10 января 2024,https://silkwaychina.ru/news/news/statya/vse-c...,Доставка из Китая может показаться сложной зад...,<p>Доставка из Китая может показаться сложной ...


#### https://prc.today/

telethon + request

In [ ]:
from telethon import TelegramClient
import pandas as pd
import asyncio
import re

In [83]:
def extract_data(messages, channel_name):
    
    channel_data = pd.DataFrame()

    for current_message in messages:

        group = '@' + channel_name

        sender_id = current_message.sender_id

        message_id = current_message.id

        datetime = current_message.date.strftime('%Y-%m-%d %H:%M:%S')

        text = current_message.text

        message_data = pd.DataFrame([[group, sender_id, message_id, datetime, text]], columns = ['group', 'sender_id', 'message_id', 'datetime', 'views', 'shares', 'text'])

        channel_data = pd.concat([channel_data, message_data], axis=0)


    channel_data = channel_data.reset_index().drop('index', axis=1)

    channel_data.to_excel(f'{channel_name}.xlsx')

async def main():
    
    api_id = 
    api_hash = ''
    phone_number = ''

    client = TelegramClient('main', api_id, api_hash, system_version="1.01")

    await client.start()

    if not client.is_connected():
        await client.connect()

    channel_names = ['PRC_TODAY']

    for channel_name in channel_names:

        channel = await client.get_entity(channel_name)

        messages = [msg async for msg in client.iter_messages(channel)]

        extract_data(messages, channel_name='prc_today_tg')

    await client.log_out()

In [ ]:
# Парсинг тг канала

await main()

In [258]:
data = pd.DataFrame()

source = 'prc_roday'

# Парсинг статей по их ссылкам из тг канала. Т.к. в нем не полные тексты, но присутствуют ссылки на них. Из исходного сайта не удается получить данные
tg = pd.read_excel('prc_today_tg.xlsx', index_col=0)

tg = tg[tg.text.notnull()]

for index, row in tg.iterrows():
    
    try:
        urls = re.findall(r'https?://[^\s]+', row['text'])
    
        # Получаем последний URL
        last_url = urls[-1] if urls else None
    
        r = requests.get(last_url)
        
        soup = BeautifulSoup(r.text, 'lxml')
        
        elements = soup.find_all('p')
        
        for i, v in enumerate(elements):
            if 'подготовлена Порталом' in v.text:
                break
    
        title = row['text'].split('\n')[2]
    
        date = row['datetime']
        
        text = ''.join([i.text for i in elements[:i]])
        
        href = np.nan
    
        sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
    
        data = pd.concat([data, sub_data], axis=0)
        break
        time.sleep(1)
        
    except Exception as e:
        print(e)
        
    break
        
#data.to_excel(f'{source}.xlsx')

In [170]:
print(data.shape)
display(data)

(3722, 5)


,source,title,date,href,text
0,prc_roday,Экономика Китая устойчива в первом квартале,2024-04-03 16:41:43,https://www.india-briefing.com/news/top-sector...,Экономика Китая приветствовала уверенный старт...
0,prc_roday,Добыча угля в Китае достигла исторического мак...,2024-04-03 14:40:27,https://www.india-briefing.com/news/top-sector...,Добыча угля в Китае: объемы производства и имп...
0,prc_roday,"Компания Huawei заявляет о росте выручки на 9,...",2024-04-03 12:39:37,https://www.india-briefing.com/news/top-sector...,"Китайская компания Huawei сообщила 29 марта, ч..."
0,prc_roday,Немецкая оптическая компания ZEISS открывает ц...,2024-04-03 10:39:21,https://www.india-briefing.com/news/top-sector...,"ZEISS, немецкая транснациональная компания, ра..."
0,prc_roday,Качественное развитие китайской экономики – бл...,2024-04-03 08:43:59,https://www.india-briefing.com/news/top-sector...,Развитие китайской экономики: Высококачественн...
...,...,...,...,...,...
0,prc_roday,,2021-03-18 07:05:28,https://www.india-briefing.com/news/top-sector...,"Компания Guangzhou Asensing Technology Co Ltd,..."
0,prc_roday,,2021-03-17 17:39:53,https://www.india-briefing.com/news/top-sector...,"Компания Guangzhou Asensing Technology Co Ltd,..."
0,prc_roday,,2021-03-17 17:37:38,https://www.india-briefing.com/news/top-sector...,"Компания Guangzhou Asensing Technology Co Ltd,..."
0,prc_roday,,2021-03-17 15:21:48,https://www.india-briefing.com/news/top-sector...,"Компания Guangzhou Asensing Technology Co Ltd,..."


#### https://russian.news.cn/china/economic.htm

webdriver + requests

In [185]:
data = pd.DataFrame()

source = 'russian.news.cn'

driver = webdriver.Chrome()

base_url = "https://russian.news.cn/china/economic.htm"

driver.get(base_url)

time.sleep(3)

#Листаем страницу до конца
i = 0
while i < 15:
    driver.find_element(By.ID, 'moreBtn').click()
    time.sleep(1)
    i+=1

items = driver.find_elements(By.CSS_SELECTOR, '.list-item') + driver.find_elements(By.CSS_SELECTOR, '.clearfix')

for item in items:
    
    try:

        title = item.find_element(By.TAG_NAME, 'a').text

        description = item.text

        href = item.find_element(By.TAG_NAME, 'a').get_attribute('href')

        r = requests.get(href)
        
        soup = BeautifulSoup(r.text, 'lxml')
        
        date = soup.find('div', class_='info').text.strip().split('丨')[0]
        
        text = ' '.join([p.text for p in soup.find_all('p')][:-1])
        
        text_with_tags = ''.join([str(p) for p in test.find_all(["p", "li", "h1", "h2", "h3", "h4", "h5", "h6"])][:-1])
    
        sub_data = pd.DataFrame([[source, title, date, href, description, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'description', 'text', 'text_with_tags'])
    
        data = pd.concat([data, sub_data], axis=0)
        
    except Exception as e:
        print(e)

# Закрыть браузер после выполнения
driver.quit()

# Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

'NoneType' object has no attribute 'text'
Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF75A691F52+60322]
	(No symbol) [0x00007FF75A60CEC9]
	(No symbol) [0x00007FF75A4C7EBA]
	(No symbol) [0x00007FF75A517676]
	(No symbol) [0x00007FF75A51773C]
	(No symbol) [0x00007FF75A50AEEC]
	(No symbol) [0x00007FF75A53C25F]
	(No symbol) [0x00007FF75A50ADB6]
	(No symbol) [0x00007FF75A53C430]
	(No symbol) [0x00007FF75A55BC80]
	(No symbol) [0x00007FF75A53BFC3]
	(No symbol) [0x00007FF75A509617]
	(No symbol) [0x00007FF75A50A211]
	GetHandleVerifier [0x00007FF75A9A94AD+3301629]
	GetHandleVerifier [0x00007FF75A9F36D3+3605283]
	GetHandleVerifier [0x00007FF75A9E9450+3563680]
	GetHandleVerifier [0x00007FF75A744326+790390]
	(No symbol) [0x00007FF75A

In [186]:
print(data.shape)
display(data)

(176, 7)


,source,title,date,href,description,text,text_with_tags
0,russian.news.cn,Китайская робототехническая компания и FAW-Vol...,2024-07-02 22:21:31,https://russian.news.cn/20240702/226c90aab40e4...,Китайская робототехническая компания и FAW-Vol...,"Шэньчжэнь, 2 июля /Синьхуа/ -- Компания UBTECH...",<h1>\r\n Ви...
0,russian.news.cn,Китай и Швейцария начнут переговоры по обновле...,2024-07-02 19:23:31,https://russian.news.cn/20240702/68906c8c41c84...,Китай и Швейцария начнут переговоры по обновле...,"Пекин, 2 июля /Синьхуа/ -- Китай и Швейцария в...",<h1>\r\n Ви...
0,russian.news.cn,В июне 2024 года китайский логистический секто...,2024-07-02 19:23:30,https://russian.news.cn/20240702/671257fe766d4...,В июне 2024 года китайский логистический секто...,"Пекин, 2 июля /Синьхуа/ -- В июне этого года в...",<h1>\r\n Ви...
0,russian.news.cn,"В Китае начинает работу первый в стране ""зелен...",2024-07-02 19:08:30,https://russian.news.cn/20240702/a78a1c50a8204...,"В Китае начинает работу первый в стране ""зелен...","Пекин, 2 июля /Синьхуа/ -- Китайская националь...",<h1>\r\n Ви...
0,russian.news.cn,Новый авиамаршрут связал Шанхай с французским ...,2024-07-02 17:06:30,https://russian.news.cn/20240702/72c0bdcb12b84...,Новый авиамаршрут связал Шанхай с французским ...,"Шанхай, 2 июля /Синьхуа/ -- Самолет с более че...",<h1>\r\n Ви...
...,...,...,...,...,...,...,...
0,russian.news.cn,,2024-06-20 20:09:50,https://russian.news.cn/20240620/d7ea84c142824...,,"Пекин, 20 июня /Синьхуа/ -- В ходе своего анти...",<h1>\r\n Ви...
0,russian.news.cn,,2024-06-20 20:16:15,https://russian.news.cn/20240620/c7dcfc8f65804...,,"Пекин, 20 июня /Синьхуа/ -- В ходе своего анти...",<h1>\r\n Ви...
0,russian.news.cn,,2024-06-20 19:49:15,https://russian.news.cn/20240620/42ae10eaf4084...,,"Пекин, 20 июня /Синьхуа/ -- В соответствии с р...",<h1>\r\n Ви...
0,russian.news.cn,,2024-06-20 18:47:15,https://russian.news.cn/20240620/732572e951ff4...,,"Пекин, 20 июня /Синьхуа/ -- Рекламная индустри...",<h1>\r\n Ви...


### businessemirates

webdriver + requests

In [187]:
data = pd.DataFrame()

source = 'businessemirates'

driver = webdriver.Chrome()

for page in list(range(1, 20)):

    base_url = f'https://businessemirates.ae/news/?page={page}'
    
    driver.get(base_url)
    
    items = driver.find_elements(By.CSS_SELECTOR, '.ib_head')
    
    for item in items:
        
        title = item.find_element(By.TAG_NAME, 'a').text
        
        href  = item.find_element(By.TAG_NAME, 'a').get_attribute('href')

        if title!='':
            
            r = requests.get(href)
            soup = BeautifulSoup(r.text, 'lxml')
            
            title = soup.find('div', class_=['full_news_head']).text
            
            description = soup.find('div', class_=['full_news_desc']).text
            
            text = soup.find('div', class_=['full_news_text']).text
            
            text_block = soup.find('div', class_=['full_news_text'])
            text_with_tags = ''.join([str(p) for p in text_block.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])][:-1])
            
            date = soup.find('div', class_=['full_news_info']).text.split('\n')[4]
        
            sub_data = pd.DataFrame([[source, title, date, href, description, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'description', 'text', 'text_with_tags'])
    
            data = pd.concat([data, sub_data], axis=0)

# Закрыть браузер после выполнения
driver.quit()

# Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

In [188]:
print(data.shape)
display(data)

(399, 7)


,source,title,date,href,description,text,text_with_tags
0,businessemirates,Власти Шарджи объявили о создании новой свобод...,02.07.2024 16:40,https://businessemirates.ae/news/uae-property-...,\nВласти Шарджи объявили о создании новой своб...,\n\nЧИТАЙТЕ НАШИ НОВОСТИ \nВ TELEGRAM-КАНАЛЕ\n...,<p></p><p>Власти Шарджи объявили о создании но...
0,businessemirates,Власти Дубая утвердили Программу увеличения ПИИ,02.07.2024 16:00,https://businessemirates.ae/news/uae-property-...,\nИсполнительный совет Дубая утвердил Программ...,\n\nЧИТАЙТЕ НАШИ НОВОСТИ \nВ TELEGRAM-КАНАЛЕ\n...,<p></p><p>Исполнительный совет Дубая утвердил ...
0,businessemirates,​Страны ОПЕК+ продлили добровольные сокращения...,02.07.2024 06:30,https://businessemirates.ae/news/uae-property-...,\nСтраны ОПЕК+ продлевают добровольные сокраще...,\n\nЧИТАЙТЕ НАШИ НОВОСТИ \nВ TELEGRAM-КАНАЛЕ\n...,"<p></p><p><span style=""background-color: initi..."
0,businessemirates,Экономику ОАЭ ожидает дальнейший рост во второ...,01.07.2024 07:40,https://businessemirates.ae/news/uae-property-...,\nЭкономику ОАЭ ожидает дальнейший рост во вто...,\n\nЧИТАЙТЕ НАШИ НОВОСТИ \nВ TELEGRAM-КАНАЛЕ\n...,<p></p><p>Во второй половине текущего года эко...
0,businessemirates,"Дом в Дубае продан за 240,5 млн дирхамов",01.07.2024 07:10,https://businessemirates.ae/news/uae-property-...,\nДом с шестью спальнями в Дубае был продан за...,\n\nЧИТАЙТЕ НАШИ НОВОСТИ \nВ TELEGRAM-КАНАЛЕ\n...,"<p></p><p>Вилла с шестью спальнями, находящаяс..."
...,...,...,...,...,...,...,...
0,businessemirates,Власти Дубая планируют привлечь более 600 новы...,22.08.2023 08:10,https://businessemirates.ae/news/uae-property-...,"\nСогласно заявлению экспертов отрасли, расшир...",\n\nЧИТАЙТЕ НАШИ НОВОСТИ \nВ TELEGRAM-КАНАЛЕ\n...,"<p></p><p>Нововведение эмирата, предпринятое н..."
0,businessemirates,Спрос на офисы в свободных зонах ОАЭ растет по...,22.08.2023 07:40,https://businessemirates.ae/news/uae-property-...,\nСвободные экономические зоны в Объединенных ...,\n\nЧИТАЙТЕ НАШИ НОВОСТИ \nВ TELEGRAM-КАНАЛЕ\n...,<p></p><p>Свободные экономические зоны ОАЭ укр...
0,businessemirates,6 крупных банков ОАЭ выделили 190 млрд дирхамо...,22.08.2023 07:10,https://businessemirates.ae/news/uae-property-...,"\nСогласно информации, предоставленной участни...",\n\nЧИТАЙТЕ НАШИ НОВОСТИ \nВ TELEGRAM-КАНАЛЕ\n...,"<p></p><p>По данным, предоставленным участника..."
0,businessemirates,ОАЭ приняли участие в совместной встрече минис...,21.08.2023 07:40,https://businessemirates.ae/news/uae-property-...,\nДелегаты из ОАЭ приняли участие в совместной...,\n\nЧИТАЙТЕ НАШИ НОВОСТИ \nВ TELEGRAM-КАНАЛЕ\n...,<p></p><p>Делегацию от ОАЭ возглавляли Его Пре...


### indianstartupnews

requests only

In [189]:
data = pd.DataFrame()

source = 'indianstartupnews'

for page in list(range(1, 50)):

        base_url = f'https://indianstartupnews.com/news?page={page}'
    
        r = requests.get(base_url)
    
        soup_all = BeautifulSoup(r.text, 'lxml')

        items = soup_all.find_all('div', class_=['col-sm-4 col-md-4 col-lg-4'])
        
        for item in items:
            
            try:
                # Находим элемент ссылки (a), содержащий нужную информацию
                link_tag = item.find('a', attrs={'aria-label': True})
                
                # Извлекаем URL из атрибута href
                href = link_tag['href'] if link_tag else None
                
                # Извлекаем текст заголовка
                title = link_tag.find('div', class_='post-title').get_text(strip=True) if link_tag else None
        
                # Находим тег <time> с классом "date"
                time_tag = soup.find('time', class_='date')
                
                # Извлекаем текст из найденного тега
                date = time_tag.get_text(strip=True) if time_tag else None
        
                url = 'https://indianstartupnews.com' + href
        
                r = requests.get(url)
                
                soup = BeautifulSoup(r.text, 'lxml')
                
                full_text = soup.find('div', class_=['article'])
                
                text_with_tags = ''.join([str(p) for p in full_text.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])])
                
                text = ''.join([p.text for p in full_text.find_all(["p"]) if p.text!=''])
        
                sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])
            
                data = pd.concat([data, sub_data], axis=0) 
                
            except Exception as e:
                    print(e)

        time.sleep(1)
            
# Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

In [190]:
print(data.shape)
display(data)

(441, 6)


,source,title,date,href,text,text_with_tags
0,indianstartupnews,Peak XV-backed AI-health startup Cloudphysicia...,None,/news/peak-xv-backed-ai-health-startup-cloudph...,Bengaluru-based AI healthcare startup Cloudphy...,<p><span>Bengaluru-based AI healthcare startup...
0,indianstartupnews,Air India to set up South Asia's largest flyin...,02 Jul 2024 18:28 IST,/news/air-india-to-set-up-south-asias-largest-...,Tata Group's airline unit Air India is set to ...,<p>Tata Group's airline unit Air India is set ...
0,indianstartupnews,Customised ayurveda beauty brand Vedix partner...,02 Jul 2024 13:48 IST,/news/customised-ayurveda-beauty-brand-vedix-p...,"Vedix, a customised Ayurvedic personal care br...","<p>Vedix, a customised Ayurvedic personal care..."
0,indianstartupnews,"RBI, ASEAN countries to create platform to fac...",02 Jul 2024 12:38 IST,/news/rbi-asean-countries-to-create-platform-t...,The Reserve Bank of India (RBI) has announced ...,<p>The Reserve Bank of India (RBI) has announc...
0,indianstartupnews,IBM signs MoU with Gujarat govt to establish A...,01 Jul 2024 21:40 IST,/news/ibm-signs-mou-with-gujarat-govt-to-estab...,US-based technology company IBM and the Depart...,<p>US-based technology company IBM and the Dep...
...,...,...,...,...,...,...
0,indianstartupnews,Amagi appoints Richard Perkett as Chief Produc...,27 Apr 2024 14:56 IST,/news/amagi-appoints-richard-perkett-as-chief-...,"Amagi, a global leader in cloud-based SaaS tec...","<p>Amagi, a global leader in cloud-based <a hr..."
0,indianstartupnews,Netcon Technologies partners with Nikitek to r...,27 Apr 2024 14:26 IST,/news/netcon-technologies-partners-with-nikite...,"Netcon Technologies, a global leader in IT inf...","<p>Netcon Technologies, a global leader in IT ..."
0,indianstartupnews,Tata Electronics partners with Pune and Bengal...,20 Apr 2024 22:50 IST,/news/tata-electronics-partners-with-pune-and-...,Tata Electronics has taken a significant step ...,<p>Tata Electronics has taken a significant st...
0,indianstartupnews,Veranda Learning announces business transfer a...,26 Apr 2024 22:41 IST,/news/veranda-learning-announces-business-tran...,Chennai-based Veranda Learning Solutions has s...,<p>Chennai-based Veranda Learning Solutions ha...


### india-briefing

requests

In [236]:
data = pd.DataFrame()

source = 'india-briefing'

for page in list(range(1, 50)):

    try:
        base_url = f"https://www.india-briefing.com/news/category/economy-trade/page/{page}/"
    
        r = requests.get(base_url)
        
        soup_all = BeautifulSoup(r.text, 'lxml')
    
        items = soup_all.find_all('div', class_=['content news mb-2rem'])
    
        for item in items:
    
            date = item.find("time", class_="entry-date")["datetime"]
            
            title = item.find("h3").text
            
            href = item.find("a")["href"]
    
            r = requests.get(href)
                    
            soup = BeautifulSoup(r.text, 'lxml')

            content = soup.find('div', class_=['article-content post-content'])
            text_with_tags = ''.join([str(p) for p in content.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])])

            text = ''.join([p.text for p in content.find_all(["p"]) if p.text!=''])
            
            sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])
                
            data = pd.concat([data, sub_data], axis=0)

        time.sleep(1)

    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
HTTPSConnectionPool(host='www.india-briefing.com', port=443): Max retries exceeded with url: /news/why-india-is-allowing-chinese-fast-fashion-giant-shein-to-re-enter-its-market-after-ban-28287.html/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000023298E9A5F0>, 'Connection to www.india-briefing.com timed out. (connect timeout=None)'))
'NoneType' object has no attribute 'find_all'
HTTPSConnectionPool(host='www.india-briefing.com', port=443): Max retries exceeded with url: /news/indias-merchandise-trade-crosses-the-us1-trillion-mark-in-2022-26958.html/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002329B3DBD90>, 'Connection to www.india-briefing.com timed out. (connect timeout=None)'))
'NoneType' object has no attribute 'find_all'
HTTPSConnectionPool(host='www.india-briefing.com', 

In [237]:
print(data.shape)
display(data)

(261, 6)


,source,title,date,href,text,text_with_tags
0,india-briefing,\nIndia’s Standing in the Emerging Asia Manufa...,2024-06-27T14:43:05+08:00,https://www.india-briefing.com/news/india-stan...,According to the Emerging Asia Manufacturing I...,<p><strong><em>According to the Emerging Asia ...
0,india-briefing,\nFDI in India FY 2023-24: Top State and Secto...,2024-06-26T15:37:11+08:00,https://www.india-briefing.com/news/fdi-landsc...,We discuss which Indian states and sectors rec...,<p><strong><em>We discuss which Indian states ...
0,india-briefing,\nIndia to Propel South Asia as Fastest Growin...,2024-06-13T20:17:36+08:00,https://www.india-briefing.com/news/india-to-p...,The World Bank projects that the Indian econom...,<p><em><strong>The World Bank projects that th...
0,india-briefing,\nIndia-Qatar Investment and Trade Relations\n,2024-06-13T19:30:33+08:00,https://www.india-briefing.com/news/india-qata...,India and Qatar have built a robust investment...,<p>India and Qatar have built a robust investm...
0,india-briefing,\nMinisterial Roles in Modi’s New Cabinet: Ful...,2024-06-11T10:30:14+08:00,https://www.india-briefing.com/news/modi-2024-...,Bharatiya Janata Party (BJP) leader Narendra M...,<p><em><strong>Bharatiya Janata Party (BJP) le...
...,...,...,...,...,...,...
0,india-briefing,\nWhat’s in India’s 2019 Interim Budget?\n,2019-02-05T10:12:05+08:00,https://www.india-briefing.com/news/indias-201...,With India’s general elections around the corn...,"<p><a href=""https://i0.wp.com/www.india-briefi..."
0,india-briefing,\nIndian Cities in Top 10 List of World’s Fast...,2019-01-02T21:07:23+08:00,https://www.india-briefing.com/news/india-tops...,India is home to top 10 fastest growing cities...,"<p><a href=""https://i0.wp.com/www.india-briefi..."
0,india-briefing,\nIndia Proposes Setting-up Free Trade Zones f...,2018-12-31T15:18:42+08:00,https://www.india-briefing.com/news/india-prop...,India wants to encourage Russian investments a...,"<p><a href=""https://i0.wp.com/www.india-briefi..."
0,india-briefing,\nWhere to Invest in India – New Issue of Indi...,2018-12-17T15:30:27+08:00,https://www.india-briefing.com/news/where-to-i...,The new issue of India Briefing magazine title...,<p>The new issue of India Briefing magazine ti...


### vietnam-briefing

In [250]:
data = pd.DataFrame()

source = 'vietnam-briefing'

for page in list(range(1, 15)):
    
    try:
        base_url = f"https://www.vietnam-briefing.com/news/category/economy-trade/page/{page}/"
    
        r = requests.get(base_url)
        
        soup_all = BeautifulSoup(r.text, 'lxml')
    
        items = soup_all.find_all('div', class_=['content news mb-2rem'])
    
        for item in items:
            
            date = item.find("time", class_="entry-date")["datetime"]
            
            title = item.find("h3").text
            
            href = item.find("a")["href"]
    
            r = requests.get(href)
                    
            soup = BeautifulSoup(r.text, 'lxml')
            
            content = soup.find('div', class_=['article-content'])
            text_with_tags = ''.join([str(p) for p in content.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])])
            
            text = ''.join([p.text for p in content.find_all(["p"]) if p.text!=''])
            
            sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])
                
            data = pd.concat([data, sub_data], axis=0)

        time.sleep(1)
        
    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

'NoneType' object has no attribute 'find_all'


In [251]:
print(data.shape)
display(data)

(138, 6)


,source,title,date,href,text,text_with_tags
0,vietnam-briefing,\nVietnam Calls for Chinese Support in Advanci...,2024-07-02T15:36:52+08:00,https://www.vietnam-briefing.com/news/vietnam-...,The Vietnam government is seeking more Chinese...,<p><em><strong>The Vietnam government is seeki...
0,vietnam-briefing,\nTrump v Biden: How Will the 2024 US Presiden...,2024-06-27T20:12:24+08:00,https://www.vietnam-briefing.com/news/trump-v-...,With both Donald Trump and sitting president J...,<p><strong><em>With both Donald Trump and sitt...
0,vietnam-briefing,\nVietnam-Mexico Trade and Investment Relations\n,2024-06-21T19:46:21+08:00,https://www.vietnam-briefing.com/news/vietnam-...,Vietnam is Mexico’s second-largest trading par...,<p>Vietnam is Mexico’s second-largest trading ...
0,vietnam-briefing,\nEU Updates Food Safety Measures for Vietnam ...,2024-06-19T18:57:45+08:00,https://www.vietnam-briefing.com/news/eu-updat...,We note the updated EU food safety regulations...,<p><strong><em>We note the updated EU food saf...
0,vietnam-briefing,\nVietnam-U.S. Trade and Investment Relations\n,2024-06-17T18:09:24+08:00,https://www.vietnam-briefing.com/news/vietnam-...,Since Vietnam and the U.S. established diploma...,<p><strong><em>Since Vietnam and the U.S. esta...
...,...,...,...,...,...,...
0,vietnam-briefing,\nESG in Vietnam: Challenges but Significant P...,2023-02-06T14:33:47+08:00,https://www.vietnam-briefing.com/news/esg-viet...,Vietnam has significant potential in the field...,<p><strong><em>Vietnam has significant potenti...
0,vietnam-briefing,\nVietnam’s Key Economic Zones: Performance an...,2023-02-01T17:29:10+08:00,https://www.vietnam-briefing.com/news/vietnam-...,Vietnam has four key economic regions (KER) in...,<p><em><strong>Vietnam has four key economic r...
0,vietnam-briefing,\nWhere are Adidas’ Factories Located in Vietn...,2023-01-31T15:54:55+08:00,https://www.vietnam-briefing.com/news/where-ad...,Vietnam is a manufacturing powerhouse when it ...,<p><strong><em>Vietnam is a manufacturing powe...
0,vietnam-briefing,\nWhere are Nike’s Factories Located in Vietna...,2023-01-27T18:39:20+08:00,https://www.vietnam-briefing.com/news/where-ar...,Vietnam is a sportswear and sports equipment m...,<p><strong><em>Vietnam is a sportswear and spo...


### middleeastbriefing

In [240]:
data = pd.DataFrame()

source = 'middleeastbriefing'

for page in list(range(1, 50)):
    
    try:
        base_url = f"https://www.middleeastbriefing.com/news/category/economy-trade/page/{page}/"
    
        r = requests.get(base_url)
        
        soup_all = BeautifulSoup(r.text, 'lxml')
    
        items = soup_all.find_all('div', class_=['content news mb-2rem'])
    
        for item in items:
    
            date = item.find("time", class_="entry-date")["datetime"]
            
            title = item.find("h3").text
            
            href = item.find("a")["href"]
    
            r = requests.get(href)
                    
            soup = BeautifulSoup(r.text, 'lxml')

            content = soup.find('div', class_=['article-content'])
            text_with_tags = ''.join([str(p) for p in content.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])])
            
            text = ''.join([p.text for p in content.find_all(["p"]) if p.text!=''])
            
            sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])
                
            data = pd.concat([data, sub_data], axis=0)
            
        time.sleep(1)

    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

In [241]:
print(data.shape)
display(data)

(402, 6)


,source,title,date,href,text,text_with_tags
0,middleeastbriefing,\nFacilitating Trade and Connectivity between ...,2024-07-01T10:25:09+00:00,https://www.middleeastbriefing.com/news/facili...,In a significant move aimed at enhancing trade...,<p>In a significant move aimed at enhancing tr...
0,middleeastbriefing,\nAbu Dhabi Courting Major UK Hedge Funds\n,2024-06-19T12:14:48+00:00,https://www.middleeastbriefing.com/news/abu-dh...,Abu Dhabi is actively courting two major UK he...,<p>Abu Dhabi is actively courting two major UK...
0,middleeastbriefing,\nDubai-Based Ports and Logistics Giant DP Wor...,2024-06-19T06:20:06+00:00,https://www.middleeastbriefing.com/news/dp-wor...,DP World is set to invest $3 billion in new po...,<p>DP World is set to invest $3 billion in new...
0,middleeastbriefing,\nQIA Reported to Acquire 10 percent Stake in ...,2024-06-12T11:11:54+00:00,https://www.middleeastbriefing.com/news/qia-re...,"Qatar’s sovereign wealth fund, QIA, is acquiri...","<p><em><strong>Qatar’s sovereign wealth fund, ..."
0,middleeastbriefing,\nChoosing Dubai as Your Investment Base and S...,2024-06-11T12:56:25+00:00,https://www.middleeastbriefing.com/news/choosi...,"Based in Dubai, IFZA is a dynamic internationa...","<p><em><strong>Based in Dubai, IFZA is a dynam..."
...,...,...,...,...,...,...
0,middleeastbriefing,\nThe Abraham Accords Are Attracting Chinese &...,2022-07-25T14:14:29+00:00,https://www.middleeastbriefing.com/news/the-ab...,Op/Ed by Chris Devonshire-Ellis They’ve tende...,<h2><em>Dubai is repositioning itself as a Gat...
0,middleeastbriefing,\nChina and Iran: Bilateral Trade Relationship...,2021-08-20T16:05:59+00:00,https://www.middleeastbriefing.com/news/china-...,Iran is an important economic and trade partne...,<p>Iran is an important economic and trade par...
0,middleeastbriefing,\nThe China-Iran 25-Year Cooperation Agreement...,2021-07-16T13:39:26+00:00,https://www.middleeastbriefing.com/news/the-ch...,We take a look back at the China-Iran 25-Year ...,<p><strong><em>We take a look back at the Chin...
0,middleeastbriefing,\nTurkey to Forge Closer Ties with ASEAN and A...,2021-01-28T16:47:21+00:00,https://www.middleeastbriefing.com/news/turkey...,Turkey is seeking to enhance ties with ASEAN a...,<li><em>Turkey is seeking to forge closer ties...


### agbi

requests

In [242]:
data = pd.DataFrame()

source = 'agbi'

for page in list(range(1, 50)):

    try:
        base_url = f'https://www.agbi.com/sectors/finance/page/{page}/'
    
        r = requests.get(base_url)
    
        soup_all = BeautifulSoup(r.text, 'lxml')
    
        items = soup_all.find_all('div', class_=['card__body'])
    
        for item in items:
    
            href = item.find_all("a")[1]["href"]
            
            title = item.find_all("a")[1].text
            
            r = requests.get(href)
                    
            soup = BeautifulSoup(r.text, 'lxml')
            
            content = soup.find('div', class_=['article__main'])
            text_with_tags = ''.join([str(p) for p in content.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])])
            
            text = ''.join([p.text for p in content.find_all(["p"]) if '[…]' not in p.text])

            header = soup.find('div', class_=['article__header'])
            
            date = header.find('time', class_=['article__date']).text
    
            sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])
    
            data = pd.concat([data, sub_data], axis=0)
            
    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

list index out of range


In [243]:
print(data.shape)
display(data)

(1067, 6)


,source,title,date,href,text,text_with_tags
0,agbi,\n\t\tMy election forecast: UK nationals in Gu...,"\n\t\tJuly 2, 2024",https://www.agbi.com/opinion/finance/2024/07/b...,“At last we can return to sanity … the mad hys...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,\n\t\tSaudi forex reserves at 18-month high in...,"\n\t\tJuly 2, 2024",https://www.agbi.com/banking-finance/2024/07/s...,Saudi Arabia’s foreign exchange reserves have ...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,\n\t\tAbu Dhabi court rulings strengthen UAE a...,"\n\t\tJuly 2, 2024",https://www.agbi.com/analysis/employment/2024/...,Recent court rulings in Abu Dhabi further tigh...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,"\n\t\tUAE to axe 2,000 procedures in ‘Zero Bur...","\n\t\tJuly 2, 2024",https://www.agbi.com/banking-finance/2024/07/u...,The Central Bank of the UAE has announced plan...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,\n\t\tGCC economic data\t,"\n\t\tJuly 2, 2024",https://www.agbi.com/economy/2024/07/gcc-econo...,The charts and tables on this page are for gen...,"<li class=""social"">\n<a class=""social__link so..."
...,...,...,...,...,...,...
0,agbi,\n\t\tWorld Bank drops growth forecast for Tun...,"\n\t\tOctober 6, 2023",https://www.agbi.com/economy/2023/10/world-ban...,Tunisia’s economic growth forecast has been do...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,\n\t\tExplained: Green bonds and how they work\t,"\n\t\tOctober 6, 2023",https://www.agbi.com/finance/2023/10/explained...,The aggregate listing value of green bonds and...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,\n\t\tSaudi economy will contract due to oil o...,"\n\t\tOctober 6, 2023",https://www.agbi.com/economy/2023/10/saudi-eco...,The World Bank has forecast that the Saudi eco...,"<li class=""social"">\n<a class=""social__link so..."
0,agbi,\n\t\tShark Tank’s O’Leary touts Abu Dhabi as ...,"\n\t\tOctober 6, 2023",https://www.agbi.com/analysis/trade/2023/10/sh...,Venture capitalist Kevin O’Leary believes Abu ...,"<li class=""social"">\n<a class=""social__link so..."


### mea-markets

requests

In [244]:
# data = pd.DataFrame()

# source = 'mea-markets'

# for page in list(range(1, 30)):

#     try:
#         base_url = f'https://www.mea-markets.com/business-news-insight/page/{page}/'
    
#         r = requests.get(base_url)
    
#         soup_all = BeautifulSoup(r.text, 'lxml')
    
#         items = soup_all.find_all('div', class_=['element_1656339709663'])
    
#         for item in items:
    
#             href = item.find_all("a")[1]["href"]
    
#             title = item.find_all("a")[1].text
            
#             r = requests.get(href)
                                
#             soup = BeautifulSoup(r.text, 'lxml')
            
#             content =  soup.find('div', class_=['article-middle-content']).find_all(["p"])
            
#             text = ''.join([i.text for i in content if i.text!=''])
            
#             date = soup.find('span', class_=['postedat']).text
    
#             sub_data = pd.DataFrame([[source, title, date, href, text]], columns=['source', 'title', 'date', 'href', 'text'])
        
#             data = pd.concat([data, sub_data], axis=0)

#     except Exception as e:
#         print(e)
    
# # Сохранение файла
# data.to_excel(f'{source}.xlsx')

In [245]:
data = pd.DataFrame()

source = 'mea-markets'

for page in list(range(1, 22)):

    base_url = f'https://www.mea-markets.com/business-news-insight/page/{page}/'

    r = requests.get(base_url)

    soup_all = BeautifulSoup(r.text, 'lxml')

    items = soup_all.find_all('div', class_=['element_1656339709663'])

    for item in items:

        href = item.find_all("a")[1]["href"]

        title = item.find_all("a")[1].text

        r = requests.get(href)

        soup = BeautifulSoup(r.text, 'lxml')
        
        content =  soup.find('div', class_=['article-middle-content'])
        text_with_tags = ''.join([str(i) for i in content.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])])

        text = ''.join([p.text for p in content.find_all(["p"]) if p.text!=''])

        date = soup.find('span', class_=['postedat']).text

        sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])

        data = pd.concat([data, sub_data], axis=0)
    
#Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

In [246]:
print(data.shape)
display(data)

(246, 6)


,source,title,date,href,text,text_with_tags
0,mea-markets,How South African Retailers Can Capitalise on ...,"Posted on July 2, 2024",https://www.mea-markets.com/how-south-african-...,"For many retailers, winter is one of the weake...","<p>For many retailers, winter is one of the we..."
0,mea-markets,Why the Promise of a Cashless Society is Key t...,"Posted on June 28, 2024",https://www.mea-markets.com/why-the-promise-of...,"By Justin Floyd, CEO of RedcloudUnder the curr...","<p class=""has-text-align-center""><strong><em>B..."
0,mea-markets,Hidden GEMs: Africa’s Frontier Markets Have Fu...,"Posted on June 27, 2024",https://www.mea-markets.com/hidden-gems-africa...,African economies have rebounded in recent mon...,<p>African economies have rebounded in recent ...
0,mea-markets,Why Nigeria’s Maritime Surveillance Systems Ne...,"Posted on June 13, 2024",https://www.mea-markets.com/why-nigerias-marit...,Nigeria will benefit much more if all agencies...,"<p class=""has-text-align-center""><strong><em>N..."
0,mea-markets,UAE Committed to Reducing Carbon Emissions in ...,"Posted on June 11, 2024",https://www.mea-markets.com/uae-committed-to-r...,In the path of achieving net-zero carbon emiss...,<p><strong>In the path of achieving net-zero c...
...,...,...,...,...,...,...
0,mea-markets,Falling Oil Prices: The Reaction of the GCC Co...,"Posted on September 14, 2015",https://www.mea-markets.com/2015-falling-oil-p...,Sharp decline in oil prices reveals the import...,"<h3><span class=""colour1"">Falling oil prices: ..."
0,mea-markets,Collective Bargaining Vital Tool for MENA,"Posted on September 8, 2015",https://www.mea-markets.com/2015-collective-ba...,The ILO brought together employer and worker r...,"<h3><span class=""colour1"">Collective Bargainin..."
0,mea-markets,The Spectre of Increasing Disputes?,"Posted on September 7, 2015",https://www.mea-markets.com/2015-regional-outl...,"By Laura Warren Partner at Clyde & Co, DohaThe...","<p class=""small""><span class=""colour1"">By Laur..."
0,mea-markets,Read more,"Posted on September 6, 2015",https://www.mea-markets.com/2015-business-conf...,According to Global Economic Conditions Survey...,"<h3><span class=""colour1"">Business Confidence ..."


### chinadaily

requests

In [247]:
data = pd.DataFrame()

source = 'chinadaily'

for page in list(range(1, 50)):

    try:
        base_url = f'https://www.chinadaily.com.cn/business/companies/page_{page}.html'
        
        r = requests.get(base_url)
        
        soup_all = BeautifulSoup(r.text, 'lxml')
        
        items = soup_all.find_all('div', class_=['mb10 tw3_01_2'])
        
        for item in items:
        
            href = item.find_all("a")[1]["href"].replace('//', 'https://')
            
            title = item.find_all("a")[1].text
        
            r = requests.get(href)
                                
            soup = BeautifulSoup(r.text, 'lxml')
            
            content =  soup.find('div', id=['Content'])
            text_with_tags = ''.join([str(i) for i in content.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])])

            text = ''.join([p.text for p in content.find_all(["p"]) if p.text!=''])
            
            date = soup.find('span', class_=['info_l']).text
            
            sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])
            
            data = pd.concat([data, sub_data], axis=0)
            
    except Exception as e:
        print(e)
    
# Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'text'
list index out of range
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
list index out of range


In [248]:
print(data.shape)
display(data)

(754, 6)


,source,title,date,href,text,text_with_tags
0,chinadaily,State Grid's efforts power up Suzhou's photovo...,\n \n chinadaily.com.cn |\n ...,https://www.chinadaily.com.cn/a/202407/02/WS66...,The State Grid Suzhou City Suburban Power Supp...,<p>The State Grid Suzhou City Suburban Power S...
0,chinadaily,Summer peak load alleviated by State Grid proj...,\n \n chinadaily.com.cn |\n ...,https://www.chinadaily.com.cn/a/202407/02/WS66...,"On June 26, the Bengbu Toupu 220 kilovolt tran...","<p>On June 26, the Bengbu Toupu 220 kilovolt t..."
0,chinadaily,PayPal bullish on nation's opening-up,\n By CHENG YU | \n China Da...,https://www.chinadaily.com.cn/a/202407/02/WS66...,China's commitment to further open up its fina...,<p>China's commitment to further open up its f...
0,chinadaily,Green compliant oilfield put into operation,\n By ZHENG XIN and LI WENFANG | \n ...,https://www.chinadaily.com.cn/a/202407/02/WS66...,China's first comprehensive environmentally fr...,<p>China's first comprehensive environmentally...
0,chinadaily,Electricity supplies put to the test ahead of ...,\n By Liu Zizheng | \n china...,https://www.chinadaily.com.cn/a/202407/01/WS66...,"At a senior high school in Xiangcheng, a count...","<p>At a senior high school in Xiangcheng, a co..."
...,...,...,...,...,...,...
0,chinadaily,Kering plans ambitious expansion,\n By ZHONG NAN in Shanghai | \n ...,https://www.chinadaily.com.cn/a/202311/16/WS65...,"Kering Group, owner of Gucci, Saint Laurent an...","<p>Kering Group, owner of Gucci, Saint Laurent..."
0,chinadaily,China opens the deepest oil and gas well in Asia,\n By Zheng Xin | \n chinada...,https://www.chinadaily.com.cn/a/202311/15/WS65...,China Petroleum and Chemical Corp struck sizab...,"<p><span style=""white-space: pre-wrap;"">China ..."
0,chinadaily,China Eastern to launch direct flights between...,\n By Wang Ying | \n chinada...,https://www.chinadaily.com.cn/a/202311/15/WS65...,China Eastern Airlines will launch direct flig...,<p>China Eastern Airlines will launch direct f...
0,chinadaily,Executives praise expo as significant platform,\n \n China Daily |\n ...,https://www.chinadaily.com.cn/a/202311/15/WS65...,HSBC confident of China's economic development...,<p><strong>HSBC confident of China's economic ...


### gbm.hsbc

In [256]:
data = pd.DataFrame()

source = 'gbm.hsbc'

driver = webdriver.Chrome()

base_url = f'https://www.gbm.hsbc.com/en-gb/insights'
    
driver.get(base_url)

time.sleep(10)

i = 0
while i < 20:
    driver.find_element(By.CLASS_NAME, 'feed-pagination___1xZaV').click()
    time.sleep(2)
    i+=1

items = driver.find_element(By.ID, 'main').find_elements(By.CLASS_NAME, 'feed__item___2bGeG')

hrefs = []
titles = []
dates = []
content_types = []

for item in items:
    try:
        hrefs.append(item.find_element(By.CLASS_NAME, 'article-card__link___qrsD2').get_attribute('href'))
        titles.append(item.find_element(By.CLASS_NAME, 'article-card__content___1h-Hz').find_element(By.CLASS_NAME, 'article-card__heading___3c3Iv').text)
        dates.append(item.find_element(By.CLASS_NAME, 'article-card__content___1h-Hz').find_element(By.CLASS_NAME, 'article-card__date___iy8zv').text)
        content_types.append(item.find_element(By.CLASS_NAME, 'article-card__content___1h-Hz').find_element(By.CLASS_NAME, 'article-type-display___2Botj').text)
    except Exception as e:
        pass
    
for href, title, date, content_type in zip(hrefs, titles, dates, content_types):
    
    try:
        if content_type == 'Article':

            r = requests.get(href)

            soup_all = BeautifulSoup(r.text, 'lxml')

            text_items = soup_all.find_all('div', class_=['content-item___1xpOP'])

            text = ''.join([item.get_text() for item in text_items if 'Disclaimer' not in item.get_text()])
            
            text_items = [i for i in soup_all.find_all('div', class_=['content-item___1xpOP'])]
            
            text_with_tags = ''

            for text_item in text_items:
                text_with_tags += ''.join([str(i) for i in text_item.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])])

            sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])

            data = pd.concat([data, sub_data], axis=0)   
            
    except Exception as e:
        pass
        
#Закрыть браузер после выполнения
driver.quit()

#Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

In [257]:
print(data.shape)
display(data)

(193, 6)


,source,title,date,href,text,text_with_tags
0,gbm.hsbc,HSBC's Business Notice Account solution optimi...,18 June 2024,https://www.gbm.hsbc.com/en-gb/insights/market...,AJ Bell is one of the largest investment platf...,<p>AJ Bell is one of the largest investment pl...
0,gbm.hsbc,Reconfiguring globalisation in a new world order,17 June 2024,https://www.gbm.hsbc.com/en-gb/insights/intern...,Globalisation is under threat. In an increasin...,<p>Globalisation is under threat. In an increa...
0,gbm.hsbc,Spotlight on India: Factors driving the markets,17 June 2024,https://www.gbm.hsbc.com/en-gb/insights/market...,India’s recognition as the world’s fastest-gro...,<p>India’s recognition as the world’s fastest-...
0,gbm.hsbc,Thriving in the never normal,17 June 2024,https://www.gbm.hsbc.com/en-gb/insights/market...,The post-pandemic world did not return to the ...,<p>The post-pandemic world did not return to t...
0,gbm.hsbc,ASEAN – a region on the rise,17 June 2024,https://www.gbm.hsbc.com/en-gb/insights/market...,The Association of Southeast Asian Nations (AS...,<p>The Association of Southeast Asian Nations ...
...,...,...,...,...,...,...
0,gbm.hsbc,"Winners and losers in a high inflation, low gr...",19 July 2022,https://www.gbm.hsbc.com/en-gb/insights/global...,Accessing Emerging MarketsExplore our latest i...,"<h2 class=""promo-card__heading___2XmwV"">Access..."
0,gbm.hsbc,Optimising cash allocation and FX decisions at...,14 July 2022,https://www.gbm.hsbc.com/en-gb/insights/innova...,Unilever is one of the world’s largest produce...,<p>Unilever is one of the world’s largest prod...
0,gbm.hsbc,Mainland China’s evolving growth drivers,12 July 2022,https://www.gbm.hsbc.com/en-gb/insights/global...,The Chinese economy is undergoing fundament...,<p> The Chinese economy is undergoing fundam...
0,gbm.hsbc,China – facing headwinds and maintaining growth,12 July 2022,https://www.gbm.hsbc.com/en-gb/insights/global...,The evolving macroeconomic environment and its...,<p>The evolving macroeconomic environment and ...


### inc42

In [ ]:
# Не пускает на сайт при перезапуске

In [ ]:
# data = pd.DataFrame()

# source = 'inc42'

# driver = webdriver.Chrome()

# base_url = f'https://inc42.com/buzz/'
    
# driver.get(base_url)

# time.sleep(30)

# i = 0
# while i < 10:
#     driver.find_element(By.ID, 'load-more-feed').click()
#     time.sleep(5)
#     i+=1

# items = driver.find_element(By.ID, 'primary').find_elements(By.CLASS_NAME, 'card-content')

In [ ]:
# hrefs = []
# titles = []
# dates = []
# items_links = []

# for item in items:
#     try:
#         hrefs.append(item.find_elements(By.CSS_SELECTOR, 'a')[1].get_attribute('href'))
#         titles.append(item.text.split('\n')[1])
#         dates.append(item.text.split('\n')[3])
#         items_links.append(item.find_elements(By.CSS_SELECTOR, 'a')[1])
#     except Exception as e:
#         pass

In [ ]:
# for href, title, date, item_link in zip(hrefs, titles, dates, items_links):

#     try:
#         driver.get(href)

#         summary = driver.find_element(By.CLASS_NAME, 'single-post-summary').text

#         text = ''.join([item.text for item in  driver.find_element(By.CLASS_NAME, 'single-post-content').find_elements(By.CSS_SELECTOR, 'p')])
        
#         sub_data = pd.DataFrame([[source, title, date, href, text, summary]], columns=['source', 'title', 'date', 'href', 'text', 'summary'])

#         data = pd.concat([data, sub_data], axis=0)   
        
#     except Exception as e:
#         pass

In [ ]:
# #Закрыть браузер после выполнения
# driver.quit()

# #Сохранение файла
# data.to_excel(f'{source}{postfix}.xlsx')

In [ ]:
# print(data.shape)
# display(data)

### kitau.ru

In [252]:
data = pd.DataFrame()

source = 'kitau'

driver = webdriver.Chrome()

base_url = f'https://kitau.ru/stati/#281'
    
driver.get(base_url)

time.sleep(5)

i=0

while i<10:
    try:
        driver.find_element(By.CLASS_NAME, 'catalog-pagination').click()
        time.sleep(2)
    except:
        pass
    
    i+=1
    
items = driver.find_elements(By.CLASS_NAME, 'articles_block2')[0].find_elements(By.CLASS_NAME, 'article_item')

for item in items:
    
    title = item.find_element(By.CLASS_NAME, 'ai_title').text

    href = item.find_element(By.TAG_NAME, 'a').get_attribute('href')

    r = requests.get(href)

    soup_all = BeautifulSoup(r.text, 'lxml')

    content = soup_all.find('div', class_=['detail_text_content'])
    
    text = ' '.join([i.text for i in content.find_all(["p"])])
    
    text_with_tags = ''.join([str(i) for i in content.find_all(["p", "li", "h2", "h3", "h4", "h5", "h6"])])
    
    date = np.nan
    
    sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])
            
    data = pd.concat([data, sub_data], axis=0)
    
# Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

In [253]:
print(data.shape)
display(data)

(48, 6)


,source,title,date,href,text,text_with_tags
0,kitau,Контейнерный кризис 2.0: почему растут ставки ...,NaN,https://kitau.ru/stati/proizvodstvo/konteynern...,\r\n\t Мировая торговля более чем на 90% завис...,<p>\r\n\t Мировая торговля более чем на 90% за...
0,kitau,Валютный контроль по внешнеторговым контрактам...,NaN,https://kitau.ru/stati/proizvodstvo/valyutnyy-...,\r\n\t С апреля 2024 года в России вступили в ...,<p>\r\n\t С апреля 2024 года в России вступили...
0,kitau,Что значит «потерять лицо» в Китае и почему эт...,NaN,https://kitau.ru/stati/proizvodstvo/chto-znach...,\r\n\t Бизнес с Китаем это не только про деньг...,<p>\r\n\t Бизнес с Китаем это не только про де...
0,kitau,Какие банки Китая принимают платежи из России,NaN,https://kitau.ru/stati/proizvodstvo/kakie-bank...,\n\n \r\n\t С февраля перечислять деньги в Кит...,<p>\n<br/>\n</p><p>\r\n\t С февраля перечислят...
0,kitau,Как зарегистрировать компанию в Пекине через W...,NaN,https://kitau.ru/stati/proizvodstvo/kak-zaregi...,\r\n\t В Пекине насчитывается около 35 тысяч и...,<p>\r\n\t В Пекине насчитывается около 35 тыся...
0,kitau,Что лучше: Кантонская ярмарка или китайский оп...,NaN,https://kitau.ru/stati/proizvodstvo/chto-luchs...,\r\n\t Выбор между посещением оптового рынка в...,"<p align=""center"" style=""text-align: left;"">\r..."
0,kitau,Где открыть бизнес в Китае: лучшие города для ...,NaN,https://kitau.ru/stati/proizvodstvo/gde-otkryt...,\r\n\t Выбор места для открытия бизнеса в Кита...,<p>\r\n\t Выбор места для открытия бизнеса в К...
0,kitau,За какие товары не принимают деньги в Китае и ...,NaN,https://kitau.ru/stati/proizvodstvo/za-kakie-t...,\r\n\t В конце февраля китайские банки начали ...,<p>\r\n\t В конце февраля китайские банки нача...
0,kitau,За что могут заморозить расчетный счет в китай...,NaN,https://kitau.ru/stati/proizvodstvo/za-chto-mo...,"\r\n\t Банки, в том числе китайские, имеют пра...","<p>\r\n\t Банки, в том числе китайские, имеют ..."
0,kitau,Как переправить большую партию товара из Китая...,NaN,https://kitau.ru/stati/proizvodstvo/kak-perepr...,\r\n\t В современном мире для людей уже не сос...,<p>\r\n\t В современном мире для людей уже не ...


### as-pacific

In [254]:
#https://as-pacific.com/blog/category-vse-o-biznese-s-kitaem/?page=2

data = pd.DataFrame()

source = 'as-pacific'

driver = webdriver.Chrome()

for page in list(range(1, 9)):
    
    base_url = f'https://as-pacific.com/blog/?page={page}'

    driver.get(base_url)
    
    time.sleep(10)
    
    items = driver.find_elements(By.CLASS_NAME, 'text-block-blog')
    
    if not items:
        time.sleep(5)
        items = driver.find_elements(By.CLASS_NAME, 'text-block-blog')
    
    titles = [item.find_element(By.TAG_NAME, 'a').text for item in items]

    hrefs = [item.find_element(By.TAG_NAME, 'a').get_attribute('href') for item in items]

    dates = [item.find_element(By.CLASS_NAME, 'date-blog').text.split(': ')[1] for item in items if item.find_element(By.CLASS_NAME, 'date-blog').text!='']
        
    for href, title, date in zip(hrefs, titles, dates):
        
        try:
            driver.get(href)
            time.sleep(1)

            text = ''.join([i.text for i in driver.find_elements(By.TAG_NAME, 'p')])
            
            tags = ["p", "li", "h2", "h3", "h4", "h5", "h6"]
            xpath_query = ' | '.join([f'//{tag}' for tag in tags])
            elements = driver.find_elements(By.XPATH, xpath_query)
            text_with_tags = [i.get_attribute('outerHTML') for i in elements]

            sub_data = pd.DataFrame([[source, title, date, href, text, text_with_tags]], columns=['source', 'title', 'date', 'href', 'text', 'text_with_tags'])

            data = pd.concat([data, sub_data], axis=0)
        
        except Exception as e:
            pass
        
# Сохранение файла
data.to_excel(f'{source}{postfix}.xlsx')

In [255]:
print(data.shape)
display(data)

(40, 6)


,source,title,date,href,text,text_with_tags
0,as-pacific,Китайские шопинг-фестивали 2022-2023,03 Августа 2023,https://as-pacific.com/blog/category-vystavki-...,Китайский Новый год или Праздник весны (春节) яв...,"[<li class=""breadcrumb-item"" itemprop=""itemLis..."
0,as-pacific,Как провести лайвстрим-шопинг в Китае,01 Августа 2023,https://as-pacific.com/blog/kak-provesti-shopi...,"В этой статье мы расскажем вам о том, как пров...","[<li class=""breadcrumb-item"" itemprop=""itemLis..."
0,as-pacific,Zhihu или китайская Quora - как использовать д...,03 Апреля 2023,https://as-pacific.com/blog/category-kitayskiy...,На сегодняшний день площадка для вопросов и от...,"[<li class=""breadcrumb-item"" itemprop=""itemLis..."
0,as-pacific,Экосистема Alibaba - набор сервисов для повыше...,21 Марта 2023,https://as-pacific.com/blog/category-vse-o-biz...,Сегодня экосистема Алибаба – одна из крупнейши...,"[<li class=""breadcrumb-item"" itemprop=""itemLis..."
0,as-pacific,Экономические тренды Китая,16 Марта 2023,https://as-pacific.com/blog/category-vse-o-biz...,"Несмотря на стремительный экономический рост, ...","[<li class=""breadcrumb-item"" itemprop=""itemLis..."
0,as-pacific,Где работать китаисту?,10 Марта 2023,https://as-pacific.com/blog/karyera-kitaistu-k...,Сегодня Китай – это важнейший торговый партнер...,"[<li class=""breadcrumb-item"" itemprop=""itemLis..."
0,as-pacific,Нишевые социальные сети Китая - как использова...,10 Марта 2023,https://as-pacific.com/blog/category-kitayskiy...,Социальных сетей и онлайн сервисов в Китае оче...,"[<li class=""breadcrumb-item"" itemprop=""itemLis..."
0,as-pacific,Цифровые экосистемы в Китае: популярные сервис...,14 Февраля 2023,https://as-pacific.com/blog/vedushchiye-tsifro...,Несмотря на сохранение антиковидных ограничени...,"[<li class=""breadcrumb-item"" itemprop=""itemLis..."
0,as-pacific,Китайский фондовый рынок: основные особенности...,01 Февраля 2023,https://as-pacific.com/blog/osobennosti-raboty...,Фондовый рынок Китая привлекает инвесторов со ...,"[<li class=""breadcrumb-item"" itemprop=""itemLis..."
0,as-pacific,Особенности работы в Китае через VPN в 2024,16 Января 2023,https://as-pacific.com/blog/vpn-v-kitaye-kakay...,При поездке в Поднебесную каждый турист обязат...,"[<li class=""breadcrumb-item"" itemprop=""itemLis..."


### Источники

In [ ]:
names = [
    'silkwaychina', #+
    'prc_roday', #+
    'russian.news.cn', #+
    'businessemirates', #+
    'india-briefing', #+
    'vietnam-briefing', #+
    'middleeastbriefing', #+
    'indianstartupnews', #+
    'mea-markets', #+
    'agbi', #+
    'chinadaily', #+
    'gbm.hsbc', #+
    'inc42', #-
    'kitau.ru', #+
    'as-pacific' #+
]